In [7]:
import os
import re
import time
import emoji
import numpy as np
import pandas as pd
import tqdm
tqdm.tqdm.pandas()

In [2]:
def clean_turkish_text(text, stoplist=None):
    """
    Türkçe metinleri temizler
    
    Args:
        text: Temizlenecek metin
        stoplist: Stopword listesi
    
    Returns:
        Temizlenmiş metin
    """
    # Küçük harfe çevir
    text = text.lower()
    
    # Yeni satırları kaldır
    text = re.sub("\n", " ", text)
    
    # URL'leri kaldır
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'www\S+', '', text)
    
    # Email'leri kaldır
    text = re.sub(r'\S+@\S+', '', text)
    
    # Emojileri kaldır
    text = emoji.replace_emoji(text)
    
    # HTML taglarını kaldır
    text = re.sub(r'<.*?>', '', text)
    
    # Sayıları kaldır
    text = re.sub(r'[0-9]+', '', text)
    
    # Özel karakterleri kaldır
    text = re.sub(r'[^\w\s]', ' ', text)
    
    # Stopword'leri kaldır
    if stoplist:
        words = text.split()
        text = ' '.join([word for word in words if word not in stoplist])
    
    # Fazla boşlukları kaldır
    text = ' '.join(text.split())
    
    return text

In [3]:
# Türkçe stopword temizliği
def load_turkish_stopwords(stopwords_path):
    """Türkçe stopword'leri yükler"""
    stoplist = []
    try:
        with open(stopwords_path, "r", encoding='utf-8') as f:
            for line in f:
                if line.strip() and line.strip()[0:1] != "#":
                    for word in line.split():
                        stoplist.append(word)
    except Exception as e:
        print(f"Stopword dosyası yüklenirken hata: {e}")
    return stoplist

In [4]:
stopwords_path = './assets/stop-words.txt'
df_path = '/mnt/d/Datasets/trnews-2025/cleaned.csv'

In [5]:
df = pd.read_csv(df_path)
df.head()

,ID,Erişim Tarihi,URL,Haber Gövdesi,Sınıf
0,00abc039d4d1,2025-07-04 16:21:30,https://www.tele1.com.tr/vedat-milor-genclik-f...,"Gastronomi yazarı Vedat Milor, sosyal medya he...",Magazin
1,1f685f4ae892,2025-07-04 16:21:30,https://coggle.it/diagram/WpMHXgULFgABtwDc/t/o...,rkel ve Sarkozy Avrupa'yı bir duraklama dönemi...,Siyaset
2,cddc437d5374,2025-07-04 16:21:31,https://www.fanatik.com.tr/takim/eskisehirspor...,Son dakika Eskişehirspor haberleri... Beşikta...,Spor
3,2ce9fc4daa5e,2025-07-04 16:21:31,https://www.haberler.com/medya/kelime-oyunu-nd...,'Kelime Oyunu'nda 'kızlı-erkekli' soru Show TV...,Magazin
4,177161f8ae91,2025-07-04 16:21:30,https://www.birgun.net/makale/kriz-mi-darbogaz...,"Kriz mi, darboğaz mı?,""ğlayan bir uygulamadır....",Finans-Ekonomi


In [9]:
df['Haber Gövdesi Cleaned'] = df['Haber Gövdesi'].progress_apply(clean_turkish_text)

100%|█████████████████████████████████████████████████████████████| 801589/801589 [11:46<00:00, 1134.60it/s]


In [10]:
df.to_csv('./data/cleaned.csv', index=False)